In [80]:
import json
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, RMSprop

In [3]:
# import the pandas dataframe
data = pd.read_pickle("processed_data.pkl")

In [4]:
data = data.astype("int32")

In [5]:
# separate the data into X and y data
# split using column names
X = data.iloc[:, 0:-2].to_numpy()
y = data.iloc[:, -1].to_numpy().transpose()

In [6]:
# shuffling with tensorflow
# indices = tf.range(start=0, limit=tf.shape(X)[0], dtype=tf.int32)
indices = tf.range(start=0, limit=tf.shape(X)[0])
shuffled_indices = tf.random.shuffle(indices)

2022-05-02 17:59:37.884841: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
shuffled_a = tf.gather(X, shuffled_indices)
shuffled_b = tf.gather(y, shuffled_indices)

In [14]:
X_shuffled = np.array(shuffled_a)
y_shuffled = np.array(shuffled_b)

In [100]:
from recent_search_milo import transpose_tweeters

transposed_y_shuffled = np.empty(shape=(len(y_shuffled)))

# change the y-shuffled's to 0-25
for i in range(len(y_shuffled)):
    transposed_y_shuffled[i] = transpose_tweeters[y_shuffled[i]]

In [101]:
print(transposed_y_shuffled)

[ 3. 12. 14. ...  8. 15.  3.]


In [102]:
# separate training and testing data
l = int(len(X_shuffled) * 0.75)

X_train = X_shuffled[:l]
X_test = X_shuffled[l:]
y_train = transposed_y_shuffled[:l]
y_test = transposed_y_shuffled[l:]

In [103]:
counter = 0

for t in X_train:
    if len(t) < 14743:
        print(counter)
    counter += 1

In [96]:
# instantiate a logistic regression
log_reg = LogisticRegression(max_iter=100)

In [29]:
# fit the logistic regression, took about 6.5 minutes to train last time
log_reg.fit(X_train, y_train)

/Users/milo/opt/anaconda3/envs/data_science/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
# evaluate score
score = log_reg.score(X_test, y_test)

In [31]:
print(score)

0.5716667824636976


In [104]:
model = Sequential()
model.add(Dense(60, input_shape = (14743,), activation = "relu"))
model.add(Dense(30, activation = "relu"))
model.add(layers.Dropout(0.2))
model.add(Dense(25, activation = "softmax"))
model.compile(Adam(lr = 0.01), "sparse_categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_101 (Dense)           (None, 60)                884640    
                                                                 
 dense_102 (Dense)           (None, 30)                1830      
                                                                 
 dropout_9 (Dropout)         (None, 30)                0         
                                                                 
 dense_103 (Dense)           (None, 25)                775       
                                                                 
Total params: 887,245
Trainable params: 887,245
Non-trainable params: 0
_________________________________________________________________


/Users/milo/opt/anaconda3/envs/data_science/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [110]:
model.fit(X_train, y_train, verbose=1, epochs=100)

Epoch 1/100
1350/1350 [==============================] - 5s 4ms/step - loss: 0.4815 - accuracy: 0.8554
Epoch 2/100
1350/1350 [==============================] - 4s 3ms/step - loss: 0.4883 - accuracy: 0.8546
Epoch 3/100
1350/1350 [==============================] - 5s 3ms/step - loss: 0.4975 - accuracy: 0.8547
Epoch 4/100
1350/1350 [==============================] - 5s 3ms/step - loss: 0.4682 - accuracy: 0.8590
Epoch 5/100
1350/1350 [==============================] - 5s 4ms/step - loss: 0.4711 - accuracy: 0.8586
Epoch 6/100
1350/1350 [==============================] - 5s 3ms/step - loss: 0.4749 - accuracy: 0.8595
Epoch 7/100
1350/1350 [==============================] - 5s 4ms/step - loss: 0.4731 - accuracy: 0.8583
Epoch 8/100
1350/1350 [==============================] - 5s 3ms/step - loss: 0.4785 - accuracy: 0.8582
Epoch 9/100
1350/1350 [==============================] - 4s 3ms/step - loss: 0.4695 - accuracy: 0.8595
Epoch 10/100
1350/1350 [==============================] - 4s 3ms/step - l

In [111]:
import cleantext as ct

def predict_tweeter(tweet: str, model: Sequential) -> str:
    # clean the tweet
    cleaned = ct.clean_tweet(tweet)
    # get vocabulary
    vocab = ct.get_word_set()
    # get counterizer
    cv = CountVectorizer(vocabulary=vocab)

    # get a word vector for it
    wv = ct.get_word_vector(tweet, cv, vocab)

    # predict on the word vector
    prediction = model.predict(wv)

    
    

